In [36]:
# email part

import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import getpass

receiver_email = "advay48@gmail.com"
sender_email = "advay@umich.edu"
#password = input("Type your password and press enter:")
password = getpass.getpass("Type your password and press enter:")

message = MIMEMultipart("alternative")
message["Subject"] = "multipart test"
message["From"] = sender_email
message["To"] = receiver_email

with open("Tracker_Output.txt", 'r') as read_file:
    text = read_file.read()
# Create the plain-text and HTML version of your message
#text = """\
"""Hi,
How are you?
Real Python has many great tutorials:
www.realpython.com"""
#html = """\
"""<html>
  <body>
    <p>Hi,
    <br>How are you?<br>
    <a href="http://www.realpython.com">Real Python</a>
    has many great tutorials.
    </p>
  </body>
</html>"""
#"""
read_file.close()
# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
#part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
#message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

Type your password and press enter:··········


In [37]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username = "advay@umich.edu"
#password = input("Type your password and press enter:")
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "imap.gmail.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

# create an IMAP4 class with SSL
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)


('OK', [b'advay@umich.edu authenticated (Success)'])

In [38]:
status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 1
# total number of emails
messages = int(messages[0])
write_file = open("Email_Input.txt", "w")

In [39]:
for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                        body_list = body.split(',')
                        body_dict = {}
                        for item in body_list:
                          item_list = item.split(':')
                          body_dict[item_list[0]] = item_list[1]

                        for item, size in body_dict.items():
                          #print((f'{item}: {size}'))
                          write_file.write((f'{item}: {size}\n'))
                          #print(file.read())

                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            """else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)"""
# close the connection and logout
write_file.close()
imap.close()
imap.logout()



Subject: 
From: Advay Singh <advay48@gmail.com>
Shirt: small, Black pants: large, tank-top: medium



('BYE', [b'LOGOUT Requested'])